In [53]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [54]:
%matplotlib notebook

#<GRADED>
import numpy as np
from numpy.matlib import repmat
import sys
import time
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets

from scipy.stats import linregress

# new torch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# # new vision-dataset-related torch imports
# import torchvision
# import torchvision.datasets as dset
# import torchvision.transforms as transforms

# misc imports
import random
#</GRADED>

Brainstorm:

1. Before Training: Use PCA to reduce the number of features (Low priority)

    1.1 Things such as the dog ID should not make a difference

2. Training: 

    2.1 Use deep neutral network to train a model and make predictions
    
    2.2 Use boosting and bagging to improve the model (Low priority)

3. Things to consider:

    3.1 Need to choose loss functions
    
    3.2 We should numericalize and normalize all features. Need to come up with ways to represent things such as "date of birth" and "N/A" data entries as numbers.
    


# STEP 1: Data Loading and Cleaning

In [55]:
# Choose from LF, LH, RF, RH 
leg = "RH"

x_file_name = leg + "_train.csv"

x_lf = pd.read_csv(x_file_name)

y_lf = x_lf[leg]



In [70]:
def clean_up_my_data_bro(x_lf, leg = "POTATO", data_type = "test"):
    
    if data_type=="test":
        x_lf = x_lf.drop(labels=["id","dob","forceplate_date"], axis=1) # these should not matter
    elif data_type=="train":
        x_lf = x_lf.drop(labels=["id","dob","forceplate_date",leg], axis=1) # these should not matter
    else:
        print("wrong data_type bro")
        
    
    # if gait is walk then 1 else -1
    for i in range(0,len(x_lf['gait'])):
        if x_lf['gait'][i] == "Walk":
            x_lf['gait'][i] = 1
        else:
            x_lf['gait'][i]= -1
    
    
    for i in range(0,len(x_lf['speed'])):
        if x_lf['speed'][i] == "Not able to walk":
            x_lf['speed'][i] = 0
        if x_lf['speed'][i] == "Not able to trot":
            x_lf['speed'][i] = -1

    x_lf = np.array(x_lf)
            
    col_remove = []
    print("x_training col number ", len(x_lf[0]))
    
    
    for j in range(0,len(x_lf[0])):

        col_sum = 0
        col_count =0
        for i in range(0,len(x_lf)):
            
            # More Cleaning Lmao
            if j==185:
                if x_lf[i][j]=="Trot":
                    x_lf[i][j]=1
            
            #if j==185: # NOT ONLY FOR 185
            if x_lf[i][j]=="Not able to trot":
                x_lf[i][j]=-1
        
            if x_lf[i][j]=="no valid trials":
                x_lf[i][j]= float("nan") # CHECK THIS
        
            if x_lf[i][j] == "No Data" or x_lf[i][j]=="no data":
                x_lf[i][j] = float("nan")


            if isinstance(x_lf[i][j], str)==True:
                #print("string found at " + str(i) + " " +str(j))
                x_lf[i][j] = float(x_lf[i][j])    


            
            if np.isnan(x_lf[i][j])==False:
                col_sum += x_lf[i][j]
                col_count += 1

            
            
    #         Idea 1: only count not nan values
    #         else:
    #             col_sum += x_lf[i][j]
    #             col_count += 1
            # Idea 2: count them but if variance is zero, remove column

        col_mean = col_sum/col_count
        col_var = 0
        
        # this loop set nan to the mean (so 0 after normalized) and calculates variance
        
        for i in range(0,len(x_lf)):
            if np.isnan(x_lf[i][j]):
                x_lf[i][j]=col_mean
            if x_lf[i][j]=="" or x_lf[i][j]==" ":
                print("an empty data entry replaced with mean")
                x_lf[i][j]=col_mean
            
            col_var += (x_lf[i][j]-col_mean)**2
            

        col_std = (col_var/float(col_count))**(1/2)

        if col_std == 0 or col_var==0:
            col_remove.append(j) 
            col_std = 1
            col_var = 1
            print("removed column ", j, " bc all data are the same")

        x_lf[:,j] -= col_mean
        x_lf[:,j] /= col_std
    
    
    # I MADE THE APPROXIMATION THAT NAN=-1, CAN IMPROVE LATER BY LOOKING AT SPECIFIC COLUMNS
    

    for i in col_remove:
        x_lf = np.delete(x_lf,obj=i,axis=1)

    print("x_lf (or t_lf) has " + str(len(x_lf[0])) + " columns")


    x_lf2 = x_lf # Cleaned_Data


    return x_lf2


In [71]:
# for i in x_lf.columns:
#     print(i)

In [72]:
x_lf2 = clean_up_my_data_bro(x_lf, leg=leg, data_type = "train")

x_training col number  367
removed column  3  bc all data are the same
removed column  185  bc all data are the same
x_lf (or t_lf) has 365 columns


<ipython-input-70-9397b69eda80>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['gait'][i] = 1


In [59]:
print(x_lf2[76][40])

4.556010570075541


In [75]:
x_lf2 = np.clip(x_lf2,-2,2)

# columns_to_plot = 50

# x = np.arange(0,len(x_lf2))
# y = x_lf2[:,columns_to_plot]

# plt.scatter(x,y)
# plt.show()


In [76]:
np.shape(x_lf2)

(110, 365)

In [77]:
outstring = []

for i in x_lf2[:,100:200]:
    
    for j in i:
        outstring.append(j)
    print(outstring)
    print("\n")
    outstring = []

[-0.5022676394143532, -0.26122789313309464, 1.4507883219461464, 1.4845085117301757, -0.9226137650751777, -1.0004821480239037, -0.014378200365477255, 0.08928688629882464, -0.6348852198105973, -0.1370708427853187, 0.9956371460813102, 1.1600474849584217, -0.3950543599331448, 0.33745463432082334, -0.5152537735487319, 0.075900934746634, -1.0121623975963427, -1.1920100971001304, -1.326280941229896, -1.2596478397395474, -0.5302921906879005, -0.43138563317772355, -0.6801322853477879, -0.9870774490226241, -0.9161837537182209, -0.6452492269670642, 0.4060300328401903, 0.6699095023337482, -1.6173670319280709, -1.0563125876438464, 0.7954335384767087, 1.0836217316756056, -0.3286958449633664, -0.45440647666786094, 0.2824024029716925, 0.3425407609636879, -0.11119893675319863, -0.2568452185412419, 0.18999851714313265, 0.1767464363758735, 0.20178157808046515, -0.2162586168709303, 0.5087657074411785, -0.6331123561618921, 0.19393893208629875, 0.03148038827890887, 0.08832933872562679, -0.16927991360873268,

In [73]:
# let labels be -1 and 1


# for i in range(0,len(y_lf)):
#     if y_lf[i]==0:
#         y_lf[i]=-1

y_lf2 = np.array(y_lf)

# STEP 2: Train a Model

In [74]:
# Create a Pytorch Multilayer Perceptron (MLP) Model
#<GRADED>
class MLPNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(MLPNet, self).__init__()
        """ pytorch optimizer checks for the properties of the model, and if
            the torch.nn.Parameter requires gradient, then the model will update
            the parameters automatically.
        """
        self.input_dim = input_dim
        
        layers = []
        
        # Initialize the fully connected layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
        layers.append(self.fc1)
        layers.append(nn.ReLU())
        layers.append(self.fc2)
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        # Implement the forward pass, with ReLU non-linearities
        
#         x=torch.flatten(x,1)
        
        
    
        return self.model(x)
#</GRADED>

In [75]:
# Training function
#<GRADED>
def train_classification_model( x, y, model, num_epochs, lr=1e-1, print_freq=100):
    """Train loop for a neural network model. Please use the SGD optimizer, optim.SGD.
    
    Input:
        train_loader:    Data loader for the train set. 
                         Enumerate through to train with each batch.
        model:           nn.Model to be trained
        num_epochs:      number of epochs to train the model for
        lr:              learning rate for the optimizer
        print_freq:      frequency to display the loss
    
    Output:
        model:   nn.Module trained model
    """
    optimizer = optim.SGD(model.parameters(), lr=lr)  # create an SGD optimizer for the model parameters
    for epoch in range(num_epochs):

        # Iterate through the dataloader for each epoch
        #for batch_idx, (imgs, labels) in enumerate(train_loader):
        for i in range(len(x)):
            # imgs (torch.Tensor):    batch of input images
            # labels (torch.Tensor):  batch labels corresponding to the inputs
            
            # Implement the training loop using imgs, labels, and cross entropy loss
            
            imgs = torch.tensor(x[i].astype(np.float32))
            labels = torch.tensor(y[i].astype(np.float32)).long()
            
            
            
            optimizer.zero_grad()
            preds = model(imgs)
            loss = nn.functional.cross_entropy(preds, labels)
            #loss = mse_loss(preds, labels)
#             loss = nn.BCELoss(preds, labels)
            loss.backward()
            optimizer.step()
            
            
        if (epoch + 1) % print_freq == 0:
            print('epoch {} loss {}'.format(epoch+1, loss.item()))
    

    return model  # return trained model
#</GRADED>

In [76]:
def mse_loss(y_pred, y_true):
    square_diff = torch.pow((y_pred-y_true), 2)
    mean_error = 0.5 * torch.mean(square_diff)
    return mean_error

In [77]:
# Test function
#<GRADED>
def test_classification_model(x,y, model):
    """Tests the accuracy of the model.
    
    Input:
        test_loader:      Data loader for the test set. 
                          Enumerate through to test each example.
        model:            nn.Module model being evaluate.
        
    Output:
        accuracy:         Accuracy of the model on the test set.
    """
    # Compute the model accuracy
    
    accuracy = 0
    
    numer = 0 
    demon = 0
    
    #for batch_idx, (imgs, labels) in enumerate(test_loader):
    for i in range(len(x)):
        
        imgs = torch.tensor(x[i].astype(np.float32))
        labels = torch.tensor(y[i].astype(np.float32)).long()
        
        _, preds = torch.max(model(imgs),dim=1)
        
        #preds = np.round(preds.numpy())
        
        labels = labels.numpy()
        
        numer+= np.sum(preds.numpy() == labels)
        demon+= np.size(preds.numpy())

    accuracy = numer/demon
    
    
    return accuracy
#</GRADED>

In [78]:
test_file_name = leg + "_test.csv"

t_lf = pd.read_csv(test_file_name)

t_id_arr = np.array(t_lf['id'])

t_lf2 = clean_up_my_data_bro(t_lf, leg, data_type="test")

x_training col number  367
removed column  3  bc all data are the same
removed column  185  bc all data are the same
x_lf (or t_lf) has 365 columns


<ipython-input-68-bb1d7424b174>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['gait'][i] = 1
<ipython-input-68-bb1d7424b174>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['speed'][i] = 0


In [79]:
# Train an MLP model on MNSIT
hidden_dim = 20
num_epochs = 2000
lr = 1e-2



mlp_model = MLPNet(input_dim=365, hidden_dim=hidden_dim, output_dim=1)
print('the number of parameters', sum(parameter.view(-1).size()[0] for parameter in mlp_model.parameters()))
mlp_model = train_classification_model(x_lf2, y_lf2, mlp_model, num_epochs=num_epochs, lr=lr)
# avg_test_acc = test_classification_model(test_loader, mlp_model)






the number of parameters 7341


IndexError: Target 1 is out of bounds.

In [80]:
# t_lf2= torch.tensor(t_lf2.astype(np.float32))

pred_lf = mlp_model(t_lf2)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [81]:
print(y_lf2)

[0 1 0 1 0 0 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0
 0 1 0 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1
 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0]


In [82]:
pred_lf

NameError: name 'pred_lf' is not defined

In [83]:
# final_lf  = np.sign(pred_lf.detach().numpy())

# for i in range(0,len(final_lf)):
#     final_lf[i]=int(float(final_lf[i]))
#     if final_lf[i]==-1:
#         final_lf[i]=0

In [84]:
final_lf = final_lf.astype(int)

NameError: name 'final_lf' is not defined

# STEP 3: Make Output Files

In [85]:
import csv

rows = zip(t_id_arr, final_lf)


NameError: name 'final_lf' is not defined

In [86]:
# Write the tuples to a CSV file

csv_file_name = leg + "_test_labels.csv"

with open(csv_file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', leg]) # write header row
    writer.writerows(rows)

NameError: name 'rows' is not defined

In [88]:
def make_sub():

    """
    This is a file that will help you convert your individual predictions to the final prediction. 
    In the same directory as this file, you should have the following 4 files:
      - LF_test_labels.csv - with at least two columns, 'id' and 'LF'
      - LH_test_labels.csv - with at least two columns, 'id' and 'LH'
      - RF_test_labels.csv - with at least two columns, 'id' and 'RF'
      - RH_test_labels.csv - with at least two columns, 'id' and 'RH'

    Running this script will convert these four files into a single CSV file, submission.csv, by
    mutating the IDs so that they also include the leg that is being checked.
    """

    legs = ["LF", "LH", "RF", "RH"]

    dfs = []

    for leg in legs:
        # read in the file
        test_prediction = pd.read_csv(f"{leg}_test_labels.csv")
        # append the abbreviation for the leg
        test_prediction['id'] = test_prediction['id'].astype(str) + f"_{leg}"
        # rename the label column
        test_prediction['label'] = test_prediction[leg]
        # exclude any potential additional columns
        dfs.append(test_prediction[['id', 'label']])

    final_df = pd.concat(dfs)
    final_df.to_csv("submission.csv", index=False)



In [89]:
# make_sub()